## Project Objectives

- Objective: hourly energy demand
- Target: load, price
- Type: day ahead (24h)

## Datasets

1. energy_dataset:

    Power system (actuals by hour)
    - time — Timestamp of the delivery hour (usually ISO 8601). Use UTC internally; handle DST carefully.

    - generation biomass — Electricity generated from biomass, in MW (instantaneous average over the hour).

    - generation fossil brown coal/lignite — Lignite-fired generation, MW.

    - generation fossil coal-derived gas — Coal-gas derived generation, MW.

    - generation fossil gas — Natural gas generation, MW.

    - generation fossil hard coal — Hard/bituminous coal generation, MW.

    - generation fossil oil — Oil-fired generation, MW.

    - generation fossil oil shale — Oil-shale–based generation, MW.

    - generation fossil peat — Peat-fueled generation, MW.

    - generation geothermal — Geothermal generation, MW.

    - generation hydro pumped storage aggregated — Net pumped-storage generation (aggregate), MW (positive = generating).

    - generation hydro pumped storage consumption — Pumped-storage pumping load/consumption, MW (positive = consuming).

    - generation hydro run-of-river and poundage — Run-of-river hydro generation, MW.

    - generation hydro water reservoir — Reservoir hydro generation, MW.

    - generation marine — Tidal/wave (marine) generation, MW.

    - generation nuclear — Nuclear generation, MW.

    - generation other — Other unspecified generation sources, MW.

    - generation other renewable — Other renewables not listed elsewhere (e.g., biomass waste-to-energy if categorized so), MW.

    - generation solar — Solar PV generation, MW.

    - generation waste — Waste-to-energy generation, MW.

    - generation wind offshore — Offshore wind generation, MW.

    - generation wind onshore — Onshore wind generation, MW.

    Forecasts (known at/ before the forecast time)

    - forecast solar day ahead — Day-ahead solar generation forecast, MW for each hour of the next day (feature; avoid using realized future solar).

    - forecast wind offshore day ahead (your list says “offshore eday ahead”; assume a typo) — Day-ahead offshore wind forecast, MW.

    - forecast wind onshore day ahead — Day-ahead onshore wind forecast, MW.

    - total load forecast — System load forecast made day-ahead or intra-day, MW (feature; do not use future actuals).

    - price day ahead — Day-ahead market price for the delivery hour, EUR/MWh (can be negative).

    Load & price targets
    - total load actual — Realized system load (demand), MW.

    - price actual — Realized/settlement price (if provided; may be balancing or intraday realized), EUR/MWh.

2. Weather (typically hourly, by city/area)
    - dt_iso — Weather observation/forecast timestamp (ISO 8601; confirm whether this is UTC or local from the source).

    - city_name — Name/identifier of the weather location (join key if multiple stations).

    - temp — Air temperature at 2 m, °C.

    - temp_min — Min temperature over the last/next observation window, °C (API-dependent).

    - temp_max — Max temperature over the last/next observation window, °C.

    - pressure — Atmospheric pressure at sea level, hPa.

    - humidity — Relative humidity, %.

    - wind_speed — Wind speed, typically m/s (sometimes km/h; check the data source).

    - wind_deg — Wind direction in degrees (0–360, meteorological).

    - rain_1h — Rainfall volume in the last 1 h, mm.

    - rain_3h — Rainfall volume in the last 3 h, mm.

    - snow_3h — Snowfall volume in the last 3 h, mm (water equivalent).

    - clouds_all — Cloudiness, % (0–100).

    - weather_id — Weather condition code (numeric category).

    - weather_main — High-level weather category (e.g., Rain, Snow).

    - weather_description — Human-readable description (e.g., “light rain”).

    - weather_icon — Icon code string (display-only; not predictive).

## Global Configs

In [ ]:
1+1

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# display all pandas columns
pd.set_option('display.max_columns', None)

# display first few rows of the DataFrame
pd.set_option('display.max_rows', 200)


## 1. Global EDA

In [ ]:
energy_data = pd.read_csv("/Users/shanzhonghan/Documents/Data/energy_dataset.csv", parse_dates=["time"])
weather_data = pd.read_csv("/Users/shanzhonghan/Documents/Data/weather_features.csv", parse_dates=["dt_iso"])

### 1.1 Energy dataset

In [ ]:
energy_data.head()

In [ ]:
energy_data.head(200)

In [ ]:
# Overall information
energy_data.info()

In [ ]:
# number of unique 
energy_data.nunique()

In [ ]:
# Drop null columns, constant columns, useless columns
drop_cols = ["generation fossil coal-derived gas", 
             "generation fossil oil shale", 
             "generation fossil peat", 
             "generation geothermal", 
             "generation hydro pumped storage aggregated", 
             "generation marine", 
             "generation wind offshore", 
             "forecast wind offshore eday ahead",
             "price day ahead",
             "total load forecast"
             ]

energy_data = energy_data.drop(columns=drop_cols)

In [ ]:
energy_data.info()

In [ ]:
# Convert time column to datetime type and set it as index
energy_data["time"] = pd.to_datetime(energy_data["time"], utc=True, infer_datetime_format=True).dt.floor("h")
energy_data = energy_data.set_index("time").sort_index()

In [ ]:
# duplicated rows
duplicate_rows = energy_data[energy_data.duplicated()]
print(f"Number of duplicate rows: {duplicate_rows.shape[0]}")

In [ ]:
# time continuity check
time_diff = energy_data.index.to_series().diff().dropna()
missing_intervals = time_diff[time_diff != pd.Timedelta(hours=1)]
print(f"Number of missing time intervals: {len(missing_intervals)}")

In [ ]:
# null values
energy_data.isnull().sum()

In [ ]:
# total rows
total_rows = energy_data.shape[0]
print(f"Total rows: {total_rows}")

Comparing the total number of the samples, there are only few missing features. We only need to check the target column: total load actual:

In [ ]:
# find the missing rows of the target column 'total load actual'
missing_target_rows = energy_data[energy_data["total load actual"].isnull()]

missing_target_rows

The gaps are scattered across the time range, so we apply a forward fill to impute short gaps and remove any initial missing values.

In [ ]:
print(f"total rows: {energy_data.shape[0]}, missing target rows: {missing_target_rows.shape[0]} before imputation:")
energy_data = energy_data.ffill().dropna()
energy_data.isnull().sum()

In [ ]:
print(f"Data shape after cleaning: {energy_data.shape}")

In [ ]:
# Overall information after cleaning
energy_data.info()

### 1.2 Weather

In [ ]:
weather_data[weather_data["city_name"]=="Seville"].head(10)

In [ ]:
# convert dt_iso to datetime
weather_data["time"] = pd.to_datetime(weather_data["dt_iso"], utc=True, infer_datetime_format=True).dt.floor("h")
weather_data = weather_data.drop(columns=["dt_iso"])

In [ ]:
# Overall information
weather_data.info()

In [ ]:
# number of unique
print("total rows:", weather_data.shape[0])
weather_data.nunique()

In [ ]:
weather_data["rain_1h"].unique()

In [ ]:
weather_data["rain_3h"].unique()

In [ ]:
# for the sake of simplicity, we will only keep objective column: weather_description
weather_data = weather_data.drop(columns=["weather_main", "weather_icon"])

In [ ]:
# duplicated rows in different citys

duplicate_rows_weather = weather_data[weather_data.duplicated(subset=['city_name', 'time'])]
print(f"Number of duplicate rows in weather data: {duplicate_rows_weather.shape[0]}")

# drop duplicate rows
weather_data = weather_data.drop_duplicates(subset=['city_name', 'time'])

# Check duplicates again
duplicate_rows_weather_check = weather_data[weather_data.duplicated(subset=['city_name', 'time'])]
print(f"Number of duplicate rows in weather data after dropping duplicates: {duplicate_rows_weather_check.shape[0]}")

In [ ]:
# set time column as index
weather_data = weather_data.set_index("time").sort_index()

In [ ]:
# time continuity check for each city
missing_counts = (weather_data
    .groupby('city_name')
    .apply(lambda g: g.index.to_series().diff().dropna().ne(pd.Timedelta('1H')).sum())
)
print(missing_counts)

In [ ]:
# null values
weather_data.isnull().sum()

In [ ]:
# check object columns in details
object_cols = weather_data.select_dtypes(include=['object']).columns
for col in object_cols:
    print(f"Value counts for column '{col}':")
    print(weather_data[col].value_counts())
    print()

Encoding categorical cols plan:
- city_name: one-hot
- weather_description: sentence-transformer


In [ ]:
# statistical summary
weather_data.describe()

**Key Findings:**

- Pressure shows both low and extreme high outliers;
- wind speed has a heavy upper tail; 
- rain/snow are zero-inflated. 

We’ll confirm with plots next.

### 1.3 Visualization

#### 1.3.1 energy data visualization

In [ ]:
# Load & Price: Overall Trend (Hourly → Daily Mean, Denoised)

daily = energy_data[['total load actual','price actual']].resample('D').mean()

fig, ax1 = plt.subplots(figsize=(12,4))
ax1.plot(daily.index, daily['total load actual'], label='Load (daily mean)')
ax1.set_ylabel('MW')
ax1.set_title('System Load (daily average)')
ax1.grid(True, alpha=0.3)

fig, ax2 = plt.subplots(figsize=(12,4))
ax2.plot(daily.index, daily['price actual'], label='Price actual (daily mean)')
ax2.set_ylabel('EUR/MWh')
ax2.set_title('Price actual (daily average)')
ax2.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Load & Price: Dual-Axis View (Hourly Detail)

fig, ax1 = plt.subplots(figsize=(12,4))
ax1.plot(energy_data.index, energy_data['total load actual'], alpha=0.5)
ax1.set_ylabel('Load (MW)')
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()
ax2.plot(energy_data.index, energy_data['price actual'], alpha=0.5)
ax2.set_ylabel('Price actual (EUR/MWh)')
ax1.set_title('Load vs Price(hourly)')
plt.show()


In [ ]:
# Seasonality – Monthly

monthly = energy_data['total load actual'].resample('MS').mean()
fig, ax = plt.subplots(figsize=(12,3.5))
ax.plot(monthly.index, monthly.values)
ax.set_title('Monthly mean load')
ax.set_ylabel('MW'); ax.grid(True, alpha=0.3)

In [ ]:
# Weekday–Hour Seasonality (Grouped by Hour)

gp = energy_data['total load actual'].groupby([energy_data.index.dayofweek,
                                               energy_data.index.hour]).mean().unstack()
fig, ax = plt.subplots(figsize=(10,4))
im = ax.imshow(gp, aspect='auto')
ax.set_title('Load by weekday (0=Mon) and hour')
ax.set_xlabel('Hour'); ax.set_ylabel('Weekday')
fig.colorbar(im, ax=ax, shrink=0.8)
plt.show()


# Price: Weekday–Hour Seasonality (Grouped by Hour)

gp = energy_data['price actual'].groupby([energy_data.index.dayofweek,
                                               energy_data.index.hour]).mean().unstack()
fig, ax = plt.subplots(figsize=(10,4))
im = ax.imshow(gp, aspect='auto')
ax.set_title('Price by weekday (0=Mon) and hour')
ax.set_xlabel('Hour'); ax.set_ylabel('Weekday')
fig.colorbar(im, ax=ax, shrink=0.8)
plt.show()



In [ ]:
# load distribution
yL = energy_data["total load actual"]
yL.plot(kind="hist", bins=60, alpha=.7, title="Load distribution"); plt.show()

In [ ]:
# price distribution
yP = energy_data["price actual"]
yP.plot(kind="hist", bins=60, alpha=.7, title="Price distribution"); plt.show()
print("Price < 0 ratio:", (yP<0).mean())


In [ ]:
def plot_acf_clean(s, max_lag=200, title="", highlight=(), daily=False):
    s = s.dropna()
    if daily:
        s = s.resample("D").mean()
    fig, ax = plt.subplots(figsize=(7,4))
    plot_acf(s, lags=max_lag, ax=ax, zero=False, alpha=0.05)
    ax.set_title(title)
    ax.set_xlim(0, max_lag)
    ax.set_ylim(-1.0, 1.0)
    ax.grid(True, alpha=0.3)
    for k in highlight:
        if k <= max_lag:
            ax.axvline(k, color="k", linestyle="--", linewidth=0.8, alpha=0.6)
            ax.text(k+1, 0.88, str(k), fontsize=9, alpha=0.7)
    plt.tight_layout()
    plt.show()





In [ ]:
# import plot_acf

from statsmodels.graphics.tsaplots import plot_acf
# Load: ACF (Hourly)
plot_acf_clean(yL, max_lag=200, title="ACF — Load (hourly)",
               highlight=(1,2,3,24,48,168), daily=False)


action: load lags: 1, 2, 3, 24, 25, 48, 168 + rolling stats: 24, 168 on shift (1)

In [ ]:
# Price: ACF (Daily Mean)
plot_acf_clean(yP, max_lag=200, title="ACF — Price (daily mean)",
               highlight=(7,14,28), daily=True)

actions: price daily lags: 1, 2, 7, 14, 28 + rolling stats: 7, 14, 28 on shift(1)

In [ ]:
# Generation Mix (Daily Mean): 

gen_cols = [c for c in energy_data.columns if c.startswith('generation ') and
            'pumped storage consumption' not in c]  # 
# Daily mean to reduce visual noise
gen_daily = energy_data[gen_cols].resample('D').mean()
gen_smooth = gen_daily.rolling(window=7, min_periods=1).mean()

order = gen_smooth.mean().sort_values(ascending=False).index
labels = [c.replace('generation ', '') for c in order]

fig, ax = plt.subplots(figsize=(12,5))
ax.stackplot(gen_smooth.index, gen_smooth[order].T.values, labels=labels)
ax.set_title('Generation Mix (7-day Mean)')
ax.set_ylabel('MW'); ax.grid(True, alpha=0.3)
ax.legend(ncols=1, fontsize=8, loc='center left', bbox_to_anchor=(1.01, 0.5))
plt.tight_layout()
plt.show()


In [ ]:
# Generation by Source (30-day Mean): Small Multiples

import matplotlib.dates as mdates

# 1) Prepare series: daily mean + 30-day rolling mean; order by long-run average
gen_cols = [c for c in energy_data.columns
            if c.startswith('generation ') and 'pumped storage consumption' not in c]
gen_daily  = energy_data[gen_cols].resample('D').mean()
gen_roll30 = gen_daily.rolling(30, min_periods=1).mean()
order = gen_roll30.mean().sort_values(ascending=False).index
titles = [c.replace('generation ', '') for c in order]

# 2) Small multiples: share the same X axis across plots, keep independent Y axes
n = len(order); ncols = 1
nrows = (n + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(10, 2*nrows), sharex=True)  # sharex=True !
axes = axes.ravel()

xmin, xmax = gen_roll30.index.min(), gen_roll30.index.max()
locator = mdates.YearLocator()       
fmt     = mdates.DateFormatter('%Y-%m')

for i, col in enumerate(order):
    ax = axes[i]
    ax.plot(gen_roll30.index, gen_roll30[col], linewidth=1.0)
    ax.set_title(titles[i], fontsize=9)
    ax.set_xlim(xmin, xmax)          
    ax.grid(True, alpha=0.25)
    if i % ncols == 0:
        ax.set_ylabel('MW')          
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(fmt)

# 3) Hide X labels on inner plots; show them only on the bottom row
for ax in axes:
    ax.label_outer()

# Turn off any unused subplots (when grid has extra slots)
for j in range(i+1, len(axes)):
    axes[j].axis('off')

fig.suptitle('Generation by source (30-day mean)', y=1.02, fontsize=12)
plt.tight_layout()
plt.show()


**Findings**

- Seasonality: Strong annual cycle and clear weekday–hour pattern in load; solar peaks in summer, near-zero in winter.

- Load–Price relation: Co-movement is visible, but price is much more volatile with occasional spikes.

- Renewables: Onshore wind changes a lot from week to week. Solar is smaller overall but very seasonal.

- Fossil Gas/coal： average level can shift for several weeks or months.

- Hydro: Reservoir and run-of-river plants smooth peaks and valleys and show clear seasons.


#### 1.3.2 weather data visualization

In [ ]:
weather_data.head()

In [ ]:
# visualization of temperature data
plt.figure(figsize=(12, 8))
plt.subplot(3, 2, 1)
sns.boxplot(x='city_name', y='temp', data=weather_data)
plt.title('Temperature Distribution by City')
plt.tight_layout()
plt.show()      

In [ ]:
# Visualization of pressure data
plt.figure(figsize=(12, 8))
plt.subplot(3, 2, 1)
sns.boxplot(x='city_name', y='pressure', data=weather_data)
plt.title('Pressure Distribution by City')
plt.tight_layout()
plt.show()

Outliers: the pressure unit might be wrong

In [ ]:
# visualization of wind speed data
plt.figure(figsize=(12, 8))
plt.subplot(3, 2, 1)
sns.boxplot(x='city_name', y='wind_speed', data=weather_data)
plt.title('Wind Speed Distribution by City')
plt.tight_layout()
plt.show()

In [ ]:
# visualization of humidity data
plt.figure(figsize=(12, 8))
plt.subplot(3, 2, 1)
sns.boxplot(x='city_name', y='humidity', data=weather_data)
plt.title('Humidity Distribution by City')
plt.tight_layout()
plt.show() 

In [ ]:
weather_data.head()

In [ ]:
# visualization of rain_1h data
plt.figure(figsize=(12, 8))
plt.subplot(3, 2, 1)
sns.boxplot(x='city_name', y='rain_1h', data=weather_data)
plt.title('Rain 1h Distribution by City')
plt.tight_layout()
plt.show()

In [ ]:
weather_data["rain_1h"].describe()

In [ ]:
# visualization of rain_3h data
plt.figure(figsize=(12, 8))
plt.subplot(3, 2, 1)
sns.boxplot(x='city_name', y='rain_3h', data=weather_data)
plt.title('Rain 3h Distribution by City')
plt.tight_layout()
plt.show()

In [ ]:
# visualization of snow_3h data
plt.figure(figsize=(12, 8))
plt.subplot(3, 2, 1)
sns.boxplot(x='city_name', y='snow_3h', data=weather_data)
plt.title('Snow 3h Distribution by City')
plt.tight_layout()
plt.show()

In [ ]:
# ---- Zero Rain/Snow Analysis ----
def zero_ratio(s):
    return (s==0).mean()

zr = weather_data.groupby('city_name').agg(
    rain1_zero_rate=('rain_1h', zero_ratio),
    rain3_zero_rate=('rain_3h', zero_ratio),
    snow3_zero_rate=('snow_3h', zero_ratio),
).sort_values(['rain1_zero_rate', 'rain3_zero_rate', 'snow3_zero_rate'], ascending=False)

ax = zr.plot(kind='bar', figsize=(10,4))
ax.set_ylabel('Zero ratio')
ax.set_title('Zero share by city (rain_1h, rain_3h, snow_3h)')
plt.tight_layout(); plt.show()

In [ ]:
# visualization of clouds_all data
plt.figure(figsize=(12, 8))
plt.subplot(3, 2, 1)
sns.boxplot(x='city_name', y='clouds_all', data=weather_data)
plt.title('Clouds All Distribution by City')
plt.tight_layout()
plt.show()

It doesn't rain or snow on most days across all 5 cities

### 1.4 Summary

**Key Findings**

- Load has highly correlation with lag 1, 2, 3, 24, 48, 168

- Daily price has highly correlation with date lag 1, 2, 7, 14, 28

- Seasonality: Strong annual cycle and clear weekday–hour pattern in load; solar peaks in summer, near-zero in winter.

- Load–Price relation: Co-movement is visible, but price is much more volatile with occasional spikes.

- Renewables: Onshore wind changes a lot from week to week. Solar is smaller overall but very seasonal.

- Fossil Gas/coal： average level can shift for several weeks or months.

- Hydro: Reservoir and run-of-river plants smooth peaks and valleys and show clear seasons.

- It doesn't rain or snow on most days


**Acions**
- clean weather data: rolling clipping temp, pressure, wind_speed, huminity; diff temp
- merge energy and weather data
- parse date and add time features: hour, day of week, is_weekend, month, season, sin/cos, holidays
- take load and price values of previous day at the same time as baseline 1: daily lag 24
- split trian, validation, and test
- apart from lag 24, add lag 48, lag 168 of load, 
- add rolling mean 3, rolling 12, rolling 24, rolling 168 on shift(24) of load
- apart from lag 24, add lag 48, lag 168 of price
- add rolling mean 3, rolling 12, rolling 24, rolling 168 on shift 24 of price
- do a first validation
- with all known feature to predict the load and price again, by using VAR, LSTM, TFT

## 2. Data Preprocessing

### 2.1 weather data preprocessing

In [ ]:
def roll_clip_mad(
    df: pd.DataFrame,
    col: str,
    group: str = "city_name",
    w: int = 45*24,          # window in hours (e.g., 45 days for temperature)
    k: float = 5.0,          # robustness factor (larger = more conservative)
    n_min: int = 24,         # minimum past samples required to enable clipping
    lower_phys: float | None = None,   # optional physical lower bound
    upper_phys: float | None = None,   # optional physical upper bound
) -> pd.DataFrame:
    """
    Causal rolling MAD clipping per city (does NOT overwrite the original column).

    Rule: x_clip = clip(x, median_past ± k * 1.4826 * MAD_past)
    - Uses only past values: shift(1) before rolling.
    - If past count < n_min, clipping is disabled at that timestamp.
    - Writes result to `col + "_clip"` and keeps the original column intact.
    """
    out = df.copy().sort_index()
    x = out[col].astype(float)

    # past-only series
    s_past = out.groupby(group)[col].transform(lambda s: s.shift(1))

    # rolling median and MAD per city
    med = s_past.groupby(out[group]).transform(lambda s: s.rolling(w, min_periods=1).median())
    mad = (s_past - med).abs().groupby(out[group]).transform(
            lambda s: s.rolling(w, min_periods=1).median())
    sigma = (1.4826 * mad).clip(lower=1e-6)  # avoid zero-width bands

    # enable clipping only when we have enough past data
    cnt = s_past.groupby(out[group]).transform(lambda s: s.rolling(w, min_periods=1).count())
    lo = np.where(cnt >= n_min, med - k * sigma, -np.inf)
    hi = np.where(cnt >= n_min, med + k * sigma,  np.inf)

    x_clip = x.clip(lower=lo, upper=hi)

    # optional physical bounds
    if lower_phys is not None:
        x_clip = x_clip.clip(lower=lower_phys)
    if upper_phys is not None:
        x_clip = x_clip.clip(upper=upper_phys)

    out[col + "_clip"] = x
    out[col + "_clip"] = x_clip
    return out

# Temperature (Kelvin): conservative bands for all cities
wd = roll_clip_mad(weather_data, col="temp", w=45*24, k=5, n_min=24, lower_phys=250, upper_phys=325)

# Pressure (after unit fix), shorter window
wd["pressure"] = np.where(wd["pressure"] > 2000, wd["pressure"]/1000, wd["pressure"])
wd = roll_clip_mad(wd, col="pressure", w=14*24, k=3, n_min=24, lower_phys=870, upper_phys=1100)

# wind speed
wd = roll_clip_mad(wd, col="wind_speed", w=30*24, k=3, n_min=24, lower_phys=0, upper_phys=60)

# humidity
wd = roll_clip_mad(wd, col="humidity", w=30*24, k=3, n_min=24, lower_phys=0, upper_phys=100)

# clouds_all
wd = roll_clip_mad(wd, col="clouds_all", w=30*24, k=3, n_min=24, lower_phys=0, upper_phys=100)

wd["temp_range"] = wd["temp_max"] - wd["temp_min"]

In [ ]:
wd.columns

In [ ]:
keep_wcols = ["city_name",
              "temp_clip",
              "pressure_clip",
              "wind_speed_clip",
              "humidity_clip",
              "clouds_all_clip",
              "rain_1h",
              "temp_min",
              "temp_max",
              "temp_range",] # for the sake of simplicity, only keep these weather columns

cwd = wd[keep_wcols]

In [ ]:
cwd.head() # we will upper clip rain_1h after splitting training and test sets

In [ ]:
# pivot to wide format

vars_to_expand = ['temp_clip', 'pressure_clip', 'wind_speed_clip', 'humidity_clip', "clouds_all_clip", "rain_1h", "temp_range"]
wide = cwd.pivot_table(index="time", columns="city_name", values=vars_to_expand, aggfunc="first")

new_cols = []
for col in wide.columns:
    var, city = col
    var = var.strip()
    city = city.strip()
    new_cols.append(f"{var}_city_{city}")
wide.columns = new_cols

wide_weather = wide.sort_index(axis=1)

In [ ]:
wide_weather.head()

### 2.2 Data merging

In [ ]:
# merge energy_data and wide_weather on time index
energy_data = energy_data.sort_index()
wide_weather = wide_weather.sort_index()
df = energy_data.join(wide_weather, how="left", rsuffix="_wx").sort_index()

In [ ]:
# Check df.info()
df.info()


### 2.3 Time Features Engineering

In [ ]:
# add time features
df['hour'] = df.index.hour
df['dayofweek'] = df.index.dayofweek
df['month'] = df.index.month

# add is_weekend feature
df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)

import holidays
years = pd.Index(df.index.year).unique().tolist()
spain_holidays = holidays.country_holidays("ES", years=years)
dates = pd.Series(df.index.date, index=df.index)
df["is_holiday"] = dates.isin(spain_holidays).astype("int8")

In [ ]:
df.head()

### 2.4 Feature Engineering

In [ ]:
# lag, rolling mean
 